In [1]:
import pandas as pd
import numpy as np
import json
import warnings
import spacy
warnings.filterwarnings("ignore")
import random

In [2]:
import os
os.getcwd()

'C:\\Users\\preet\\Desktop\\CAPstone\\Feature 10'

In [3]:
train_data = pd.read_csv("C:\\Users\\preet\\Desktop\\CAPstone\\train.csv")
features = pd.read_csv("C:\\Users\\preet\\Desktop\\CAPstone\\features.csv")
patient_notes = pd.read_csv("C:\\Users\\preet\\Desktop\\CAPstone\\patient_notes.csv")

In [4]:
train_data.head()

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']


In [5]:
features.head()

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded


In [6]:
patient_notes.head()

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [7]:
train_data.head()

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']


In [17]:
new_train = train_data[(train_data.feature_num == 10)][['pn_num','annotation',"location","feature_num"]]

In [18]:
new_train.pn_num.unique()

array([  16,   41,   46,   82,  100,  161,  188,  211,  224,  225,  234,
        331,  344,  352,  372,  390,  426,  464,  489,  495,  510,  575,
        583,  585,  625,  653,  669,  673,  699,  701,  719,  768,  769,
        777,  796,  804,  810,  816,  817,  821,  831,  861,  883,  970,
        977,  987, 1010, 1110, 1115, 1133, 1146, 1149, 1151, 1165, 1173,
       1186, 1200, 1209, 1235, 1250, 1264, 1270, 1281, 1318, 1376, 1512,
       1547, 1581, 1587, 1613, 1622, 1652, 1692, 1694, 1699, 1723, 1742,
       1809, 1835, 1847, 1855, 1881, 1939, 1948, 2078, 2151, 2163, 2184,
       2198, 2201, 2223, 2271, 2285, 2290, 2308, 2315, 2425, 2428, 2430,
       2436], dtype=int64)

In [19]:
new_train.head()

,pn_num,annotation,location,feature_num
10,16,['3-4 months of'],['56 69'],10
23,41,['STARTED 2-3 MO AGO'],['48 66'],10
36,46,['for 2-3 months'],['197 211'],10
49,82,"['3 months of', 'over the past 3 months']","['64 75', '187 209']",10
62,100,"['2-3 months', '2-3 months']","['111 121', '267 277']",10


In [20]:
new_train.shape

(100, 4)

#### Removing all [ ] or null columns 

In [21]:
new_train = new_train[new_train.annotation != '[]']

In [22]:
new_train.shape

(93, 4)

**Removing all [ & ] in strings**

In [23]:
new_train["annotation"] = new_train.annotation.str.replace('[','')
new_train["annotation"] = new_train.annotation.str.replace(']','')
new_train["location"] = new_train.location.str.replace('[','')
new_train["location"] = new_train.location.str.replace(']','')

In [24]:
new_train.head()

,pn_num,annotation,location,feature_num
10,16,'3-4 months of','56 69',10
23,41,'STARTED 2-3 MO AGO','48 66',10
36,46,'for 2-3 months','197 211',10
49,82,"'3 months of', 'over the past 3 months'","'64 75', '187 209'",10
62,100,"'2-3 months', '2-3 months'","'111 121', '267 277'",10


#### special row in which data was stored incorrectly need to correct it

In [26]:
new_train.loc[new_train.location == "'883 900'",'annotation'] = "'dad  heart attack'"

In [69]:
#new_train[new_train.pn_num == 1581][0:1].annotation

**splitting locations and annotations**

In [28]:
new_train = (new_train.set_index(['pn_num',"feature_num"]) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(new_train.columns, axis=1))

In [68]:
#new_train.loc[new_train.pn_num == 2436]

**Creating two new columns "start location" and "end location"**

In [30]:
new_train["start_location"] = new_train["location"].apply(lambda x: x.split()[0][1:])
new_train["end_location"] = new_train["location"].apply(lambda x: x.split()[-1][:-1])

In [31]:
new_train.head()

,pn_num,annotation,location,feature_num,start_location,end_location
0,16,'3-4 months of','56 69',10,56,69
1,41,'STARTED 2-3 MO AGO','48 66',10,48,66
2,46,'for 2-3 months','197 211',10,197,211
3,82,'3 months of','64 75',10,64,75
4,82,'over the past 3 months','187 209',10,187,209


In [32]:
row_index = []
start_location = []
end_location = []
pn_notes = set()
for index,row in new_train.iterrows():   # Iterating every row in trainn data
    if row["pn_num"] not in pn_notes:    # Verifying whether there is a change in pn_num to reinitialize start and end location lists
        start_location = []
        end_location = []
        pn_notes.add(row["pn_num"])      # Appending pn_num
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
    else: 
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])

In [33]:
print(row_index)

[28, 78]


#### The features in these row indexes have repeated start or end locations, so one of the repeated rows should be eliminated to fix this problem as this will be a issue while training spacy NER model


In [67]:
#new_train.iloc[45:47,]   # 45

In [66]:
#np.array(patient_notes.pn_history[patient_notes.pn_num == 82])[0][126:151]

In [65]:
#new_train.iloc[73:75,]   #73

In [64]:
#new_train.iloc[84:86,] # 84

In [60]:
#np.array(patient_notes.pn_history[patient_notes.pn_num == 211])[0][161:183]

In [61]:
#new_train.iloc[104:106,] # 104

In [62]:
#np.array(patient_notes.pn_history[patient_notes.pn_num == 225])[0][461:519]

In [63]:
#new_train.iloc[293:295,] # 104

#### Code to identify the drop indexes by comparing the start and end locations of the similar locations

In [34]:
row_index = []
start_location = []
end_location = []
pn_notes = set()
drop_index = []
for index,row in new_train.iterrows():   # Iterating every row in trainn data
    if row["pn_num"] not in pn_notes:    # Verifying whether there is a change in pn_num to reinitialize start and end location lists
        start_location = []
        end_location = []
        pn_notes.add(row["pn_num"])      # Appending pn_num
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
    else: 
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
            if row["start_location"] in start_location:
                if row["end_location"] >= new_train.iloc[index -1,5]:
                    drop_index.append(index -1)
                else:
                    drop_index.append(index)
            elif row["end_location"] in end_location:
                if row["start_location"] <= new_train.iloc[index -1,4]:
                    drop_index.append(index - 1)
                else:
                    drop_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])

In [35]:
len(drop_index)

2

In [36]:
print(drop_index)

[27, 78]


#### Dropping the unwanted indexes

In [37]:
new_train = new_train.drop(drop_index,axis=0)

In [59]:
#new_train.loc[(new_train.end_location == '141')]

In [38]:
#new_train = new_train.drop(1077,axis = 0)

In [ ]:
#new_train.loc[(new_train.end_location == '141')]

In [58]:
#new_train.loc[(new_train.pn_num == 2428)]

In [42]:
#new_train = new_train.drop(1123,axis = 0)

**Finding unique patient numbers in the train dataset**

In [41]:
pn_Num_list = list(new_train.pn_num.unique())

In [42]:
print(pn_Num_list)

[16, 41, 46, 82, 100, 161, 188, 211, 225, 234, 331, 344, 352, 372, 390, 426, 464, 489, 495, 510, 575, 583, 653, 669, 673, 699, 701, 719, 768, 777, 796, 810, 816, 817, 821, 831, 861, 883, 970, 977, 987, 1010, 1110, 1115, 1133, 1146, 1149, 1151, 1165, 1173, 1186, 1200, 1209, 1235, 1250, 1264, 1270, 1281, 1318, 1376, 1512, 1547, 1581, 1587, 1613, 1622, 1652, 1692, 1694, 1699, 1723, 1742, 1809, 1835, 1855, 1881, 1939, 1948, 2078, 2151, 2163, 2184, 2198, 2201, 2271, 2285, 2290, 2308, 2315, 2425, 2428, 2430, 2436]


In [43]:
final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in new_train[new_train.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_"+ str(row["feature_num"]))) # store the locations in numeric format
        
    text = patient_notes[patient_notes.pn_num == patient_number]["pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list

In [45]:
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample_Week5.json", "w") as outfile:
    outfile.write(json_object)

In [46]:
import json
 
# Opening JSON file
f = open("sample_Week5.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

#### Deviding the data into train and validation

In [48]:
len(data)

93

In [49]:
train = data[:75]
validation = data[75:]

In [50]:
len(train),len(validation)

(75, 18)

In [51]:
from spacy.tokens import DocBin
import spacy
import json
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import random

In [52]:
# converting the json file to spacy file in order to use it in training model

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [53]:
create_training(train)

100%|█████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 224.92it/s]


In [54]:
#train data set

camp_train = create_training(train)
camp_train.to_disk("nbme_train_Week5_feature10.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 75/75 [00:00<00:00, 462.04it/s]


In [55]:
#test data set

camp_train = create_training(validation)
camp_train.to_disk("nbme_validation_Week5_feature10.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 337.38it/s]


In [56]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-03-31 12:22:46.465494: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-31 12:22:46.465647: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [57]:
!python -m spacy train config.cfg --output ./output

[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     66.67    0.00    0.00    0.00    0.00
  2     200       4057.84   2359.83   30.43   25.93   36.84    0.30


2022-03-31 12:23:00.701962: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-31 12:23:00.702049: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-03-31 12:23:09,370] [INFO] Set up nlp object from config
[2022-03-31 12:23:09,386] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-31 12:23:09,401] [INFO] Created vocabulary
[2022-03-31 12:23:09,401] [INFO] Finished initializing nlp object
[2022-03-31 12:23:10,648] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


  5     400         58.75    214.58   35.00   33.33   36.84    0.35
  8     600         61.17    113.96   48.65   50.00   47.37    0.49
 10     800         76.41     95.38   38.89   41.18   36.84    0.39
 13    1000         61.23     59.62   30.30   35.71   26.32    0.30
 16    1200         71.98     44.55   35.29   40.00   31.58    0.35
 18    1400         58.71     34.49   40.00   43.75   36.84    0.40
 21    1600         71.86     25.18   34.15   31.82   36.84    0.34
 24    1800         52.31     20.54   34.29   37.50   31.58    0.34
 26    2000         23.49      8.23   33.33   35.29   31.58    0.33
 29    2200         16.59      5.27   27.03   27.78   26.32    0.27
[+] Saved pipeline to output directory
output\model-last
